In [1]:
import geopandas as gpd
import pandas as pd
from src.ecodonut.preprocessing import merge_objs_by_buffer, project_points_into_polygons, min_max_normalization
import numpy as np


def remove_nulls(x):
    if isinstance(x, list):
        x = [item for item in x if pd.notnull(item)]
        if len(x) == 0:
            return None
        if len(x) == 1:
            return x[0]
    return x


def industrial_preprocessing(gdf_OVNE, gdf_industrial):
    gdf_OVNE.to_crs(3857, inplace=True)
    gdf_OVNE = gdf_OVNE[
        gdf_OVNE['name'].str.upper().str.contains(
            'ПОЛИГОН|ЗАВОД|ТБО|ТКО|ЦЕХ|КОТЕЛЬНАЯ|РУДНИК|КАНАЛ|КАРЬЕР|СТАНЦИЯ|ПРОИЗВОД|ПРОМЫШЛЕН')]
    gdf_OVNE = gdf_OVNE.filter(items=['name', 'geometry', 'dangerous_level'])

    gdf_industrial.to_crs(3857, inplace=True)
    gdf_industrial = gdf_industrial.filter(items=['name', 'geometry'])

    gdf_industrial = merge_objs_by_buffer(gdf_industrial, 100)
    gdf_industrial['name'] = gdf_industrial['name'].apply(remove_nulls)
    union = project_points_into_polygons(gdf_OVNE, gdf_industrial)
    union[['name', 'dangerous_level']] = union.apply(lambda row: pd.Series(
        max(list(zip(row['name_right'], row['dangerous_level'])), key=lambda x: (-x[1], len(str(x[0]))))), axis=1)

    union['name'] = union.apply(lambda row: row['name'] if pd.notna(row['name']) else (
        row.name_left[0] if len(row.name_left) == 1 else row.name_left), axis=1)
    union.dropna(subset='name', inplace=True)
    union.drop(columns=['name_left', 'name_right', 'index_right'], inplace=True)
    union['type'] = 'industrial'
    union['dangerous_level'] = union['dangerous_level'].fillna(4)
    union['initial_impact'] = union.apply(lambda x: -10 if x.dangerous_level == 1 else (
        -8 if x.dangerous_level == 2 else (-6 if x.dangerous_level == 3 else -4)), axis=1)
    union['fading'] = union.apply(lambda x: 0.8 if x.dangerous_level == 1 else (
        0.6 if x.dangerous_level == 2 else (0.4 if x.dangerous_level == 3 else 0.2)), axis=1)
    union['total_impact_radius'] = 1000 * union['initial_impact'] * union['fading']
    union = gpd.GeoDataFrame(union, crs=3857)
    union = union.loc[~union['geometry'].is_empty]
    union = union[~union['geometry'].duplicated()]
    return union


city = 'Моcковская область'

In [2]:
ovne = gpd.read_parquet(f'.\\{city}\\ovne.parquet').to_crs(3857)
ovne.rename(columns={'Наименование объекта': 'name', 'Категория \nобъекта НВОС': 'dangerous_level'}, inplace=True)
industrial = gpd.read_parquet(f'.\\{city}\\industrial.parquet').to_crs(3857)

In [4]:
zhd = gpd.read_parquet(f'.\\{city}\\rails.parquet').to_crs(3857)
azs = gpd.read_parquet(f'.\\{city}\\fuel.parquet').to_crs(3857)
dumps = gpd.read_parquet(f'.\\{city}\\landfill.parquet').to_crs(3857)
roads = gpd.read_parquet(f'.\\{city}\\roads.parquet').to_crs(3857)
recreation = gpd.read_parquet(f'.\\{city}\\recreation.parquet').to_crs(3857)
nature_reserve = gpd.read_file(f'.\\{city}\\nature_reserve.geojson').to_crs(3857)

In [5]:
water_base = gpd.read_parquet(f'.\\{city}\\water.parquet').to_crs(3857)

In [7]:
forest = gpd.read_parquet(f'.\\{city}\\woods.parquet').to_crs(3857)

In [ ]:
# water_base = gpd.read_file(f'.\\{city}\\water.geojson').to_crs(3857)
# forest = gpd.read_file(f'.\\{city}\\woods.geojson').to_crs(3857)

In [5]:
# ovne = ovne.rename(columns={'Категория \nобъекта НВОС_y': 'dangerous_level'})
# ovne = ovne.rename(columns={'Наименование объекта_x': 'name'})

In [6]:
# gdf_POO = gpd.read_file(f'.\\{city}\\ПОО.geojson').to_crs(3857)
# gdf_POO['name'] = gdf_POO['name'].apply(lambda x: x.replace('\n', ' '))
# gdf_POO.rename(columns={'danger_lev': 'dangerous_level'}, inplace=True)
# gdf_POO['dangerous_level'] = gdf_POO['dangerous_level'].astype(int)
# gdf_POO['dangerous_level'] = np.where(gdf_POO['dangerous_level'] == 5, 1,
#                                       np.where(gdf_POO['dangerous_level'] == 4, 2,
#                                                np.where(gdf_POO['dangerous_level'] == 1, 3,
#                                                         np.where(gdf_POO['dangerous_level'] == 2, 3, 3))))
# ovne = pd.concat([gdf_POO, ovne])

In [8]:
ovne = ovne.filter(items=['name', 'geometry', 'dangerous_level'])

In [15]:
ovne = ovne.dropna(subset='name')
ovne = ovne.dropna(subset='dangerous_level')
ovne['dangerous_level'] = ovne['dangerous_level'].astype(int)
industrial = industrial_preprocessing(ovne, industrial)
industrial['geometry'] = industrial['geometry'].simplify(30)

In [16]:
azs = azs.filter(items=['name', 'geometry'])
azs['geometry'] = azs['geometry'].buffer(40)
azs['type'] = 'petrol station'
azs['initial_impact'] = -4
azs['total_impact_radius'] = -1000 * 4 * 0.15
azs = azs.loc[~azs['geometry'].is_empty]
azs = azs[~azs['geometry'].duplicated()]
azs['geometry'] = azs['geometry'].simplify(30)
azs.name = azs.name.fillna('Без названия')


In [17]:
dumps.to_crs(3857, inplace=True)
dumps = dumps.filter(items=['name', 'geometry'])
dumps['type'] = 'landfill'
dumps['initial_impact'] = -3
dumps = dumps.loc[~dumps['geometry'].is_empty]
dumps = dumps[~dumps['geometry'].duplicated()]
dumps['area'] = dumps.area
dumps['area_normalized'] = min_max_normalization(np.sqrt(dumps['area']), 1, 10)
dumps['total_impact_radius'] = -1000 * 3 * 0.4 * dumps['area_normalized']
dumps['geometry'] = dumps['geometry'].simplify(30)
dumps.name = dumps.name.fillna('Без названия')


In [18]:
roads_reg = roads[roads['reg'] == 2]
roads_reg = roads_reg.filter(items=['geometry'])
roads_reg.to_crs(3857, inplace=True)
roads_reg = roads_reg[~roads_reg['geometry'].duplicated()]
roads_reg['geometry'] = roads_reg['geometry'].buffer(20)
roads_reg = roads_reg['geometry'].unary_union
roads_reg = gpd.GeoDataFrame([roads_reg], columns=['geometry'], crs=32636)
roads_reg['name'] = 'Дорога регионального назначения'
roads_reg['type'] = 'regional_road'
roads_reg['initial_impact'] = -2
roads_reg['total_impact_radius'] = -1000 * 2 * 0.1
roads_reg['geometry'] = roads_reg['geometry'].simplify(30)
# roads_reg = roads_reg.explode(index_parts=False).reset_index(drop=True)

roads_fed = roads[roads['reg'] == 1]
roads_fed = roads_fed.filter(items=['geometry'])
roads_fed = roads_fed[~roads_fed['geometry'].duplicated()]
roads_fed['geometry'] = roads_fed['geometry'].buffer(40)
roads_fed = roads_fed['geometry'].unary_union
roads_fed = gpd.GeoDataFrame([roads_fed], columns=['geometry'], crs=32636)
roads_fed['name'] = 'Дорога федерального назначения'
roads_fed['type'] = 'federal_road'
roads_fed['initial_impact'] = -4
roads_fed['total_impact_radius'] = -1000 * 4 * 0.1
roads_fed['geometry'] = roads_fed['geometry'].simplify(30)
# roads_fed = roads_fed.explode(index_parts=False).reset_index(drop=True)


In [19]:
ZHD_gdf = zhd
ZHD_gdf = ZHD_gdf.filter(items=['geometry'])
ZHD_gdf = ZHD_gdf[~ZHD_gdf['geometry'].duplicated()]
ZHD_gdf['geometry'] = ZHD_gdf['geometry'].buffer(30)
ZHD_gdf = ZHD_gdf['geometry'].unary_union
ZHD_gdf = gpd.GeoDataFrame(geometry=[ZHD_gdf], crs=32636)
ZHD_gdf['name'] = 'ЖД Пути'
ZHD_gdf['type'] = 'railway'
ZHD_gdf['initial_impact'] = -3
ZHD_gdf['total_impact_radius'] = -1000 * 3 * 0.1
ZHD_gdf['geometry'] = ZHD_gdf['geometry'].simplify(30)


In [21]:
from shapely import MultiPolygon, unary_union

water = water_base.copy()
water.geometry = water.geometry.buffer(0)
# border = gpd.read_file('./Ленобласть/border.geojson').to_crs(3857)
# border = border.geometry[0]
# water = gpd.clip(water, border.buffer(0))
# water = water_base.copy()
water = water.filter(items=['name', 'water', 'geometry'])
water = water.rename(columns={'water': 'type'})

water['name'] = water['name'].fillna(value='Без названия')

water_without_name = water[water['name'] == 'Без названия']
water = water[water['name'] != 'Без названия']

intersect_mask = water_without_name.geometry.apply(
    lambda geom: water.geometry.intersects(geom).any()
)
water_without_name = water_without_name[~intersect_mask]

water = pd.concat([water, water_without_name])

In [22]:
from collections import Counter


def process_types(type_list):
    count = Counter(type_list)

    if count[None] == len(type_list):
        return None

    most_common_type = count.most_common(1)[0][0]
    return most_common_type


water = water.groupby('name').agg(list).reset_index()
water['type'] = water['type'].apply(process_types)

In [23]:
water_polygons = water[(water['type'] != 'river') & (water['type'] != 'stream') & (water['type'] != 'canal')]
water_lines = water.loc[list(set(water_polygons.index.tolist()) ^ set(water.index.tolist()))]
buf = 100
water_lines['geometry'] = water_lines['geometry'].apply(unary_union).apply(lambda x: x.buffer(buf).buffer(-buf))
water_lines = gpd.GeoDataFrame(water_lines, geometry='geometry', crs=3857).explode(index_parts=False).reset_index()
top = water_lines.area.quantile(.75)
water_lines = water_lines[water_lines.area >= top]
water_lines['name'] = water_lines['name'].apply(lambda x: list(set(x)) if isinstance(x, list) else x)
water_lines['type'] = water_lines['type'].apply(lambda x: list(set(x)) if isinstance(x, list) else x)
water_lines['total_impact_radius'] = 1000 * 3 * 0.1

buf = 400
water_polygons['type'] = water_polygons['type'].fillna('undefined water')
water_polygons['geometry'] = water_polygons['geometry'].apply(unary_union).apply(lambda x: x.buffer(buf).buffer(-buf))
water_polygons = gpd.GeoDataFrame(water_polygons, geometry='geometry', crs=3857)
water_polygons = water_polygons.explode(index_parts=False).reset_index()
top = water_polygons.area.quantile(.75)
water_polygons = water_polygons[water_polygons.area >= top]
water_polygons['name'] = water_polygons['name'].apply(lambda x: list(set(x)) if isinstance(x, list) else x)
water_polygons['type'] = water_polygons['type'].apply(lambda x: list(set(x)) if isinstance(x, list) else x)

water_polygons['area_normalized'] = min_max_normalization(np.sqrt(water_polygons.area), 1, 2)
water_polygons['total_impact_radius'] = np.round(1000 * 3 * 0.1 * water_polygons['area_normalized'])


In [24]:
def list_to_str(x):
    if isinstance(x, list):
        if len(x) == 0:
            return None
        if len(x) == 1:
            return x[0]
        return str(x).replace('[', '').replace(']', '').replace('\"', '').replace('\'', '')
    return x


all_water = pd.concat([water_polygons, water_lines])
all_water = all_water.map(list_to_str)
all_water['initial_impact'] = 3
all_water['geometry'] = all_water['geometry'].apply(lambda x: x.simplify(50))


In [25]:
all_wood = pd.concat([forest, recreation])
all_wood = all_wood.filter(items=['name', 'geometry'])
top = all_wood.area.quantile(.40)
all_wood = all_wood[all_wood.area >= top]
all_wood = all_wood['geometry'].buffer(800).unary_union.buffer(-800)
all_wood = gpd.GeoDataFrame(geometry=[all_wood], crs=3857).explode(index_parts=False)
all_wood['geometry'] = all_wood['geometry'].simplify(50)
top = all_wood.area.quantile(.75)
all_wood = all_wood[all_wood.area >= top]
all_wood['name'] = 'Зелёная зона'
all_wood['type'] = 'forest'
all_wood['layer_impact'] = 3
all_wood['source'] = False


In [26]:
nature_reserve = nature_reserve.filter(items=['name', 'geometry'])
nature_reserve['normalized_area'] = min_max_normalization(np.sqrt(nature_reserve.area), 1, 5)
nature_reserve['type'] = 'nature_reserve'
nature_reserve['geometry'] = nature_reserve['geometry'].simplify(50)
nature_reserve['fading'] = 0.2
nature_reserve['initial_impact'] = 5
nature_reserve['total_impact_radius'] = np.round(1000 * nature_reserve['initial_impact'] * nature_reserve['fading'] * \
                                                 nature_reserve['normalized_area'])


In [27]:
from ecodonut.preprocessing import calc_layer_count

industrial = gpd.GeoDataFrame(industrial, geometry='geometry', crs=3857)
azs = gpd.GeoDataFrame(azs, geometry='geometry', crs=3857)
dumps = gpd.GeoDataFrame(dumps, geometry='geometry', crs=3857)
roads_fed = gpd.GeoDataFrame(roads_fed, geometry='geometry', crs=3857)
roads_reg = gpd.GeoDataFrame(roads_reg, geometry='geometry', crs=3857)
ZHD_gdf = gpd.GeoDataFrame(ZHD_gdf, geometry='geometry', crs=3857)
all_water = gpd.GeoDataFrame(all_water, geometry='geometry', crs=3857)

all_wood = gpd.GeoDataFrame(all_wood, geometry='geometry', crs=3857)

nature_reserve = gpd.GeoDataFrame(nature_reserve, geometry='geometry', crs=3857)

all_in = gpd.GeoDataFrame(pd.concat(
    [industrial, azs, dumps, roads_reg, roads_fed, ZHD_gdf, all_water, nature_reserve], ignore_index=True),
    geometry='geometry', crs=3857)
all_in = all_in.filter(items=['name', 'type', 'initial_impact', 'total_impact_radius', 'geometry'])
all_in['layers_count'] = calc_layer_count(all_in, 2, 10)

In [28]:
from ecodonut import distribute_levels, combine_geometry

distributed_all = distribute_levels(all_in)
distributed_all = gpd.GeoDataFrame(pd.concat([distributed_all, all_wood]), geometry='geometry', crs=3857)
distributed_all = combine_geometry(distributed_all)

In [29]:
# border = gpd.read_file('./Ленобласть/border.geojson').to_crs(3857)
# border = border.geometry[0]
# distributed_all2 = gpd.clip(distributed_all, border.buffer(0))
distributed_all2 = distributed_all

In [30]:
from ecodonut.output import collections_to_str

distributed_all2 = distributed_all2.map(collections_to_str)

In [31]:
def replace_special(loc):
    if isinstance(loc, str):
        loc = loc.replace('(', '').replace(')', '').replace('[', '').replace(']', '')
        if loc.endswith(','):
            loc = loc[:-1]
    return loc


distributed_all2 = distributed_all2.map(replace_special)

In [32]:
distributed_all2.to_parquet(f'eco_karkas_{city}.parquet')
distributed_all2.to_file(f'eco_karkas_{city}.gpkg')

In [ ]:
from ecodonut import get_map

get_map(distributed_all).save(f'eco_karkas_{city}.html')

In [ ]:
distributed_all2.to_crs(3857)